<a href="https://colab.research.google.com/github/DhrumilShah3/learningreagentclojure/blob/master/humanproteinatlasimageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Data

In [ ]:
!pip install kaggle
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision import *
from fastai.metrics import error_rate
import pandas as pd
import numpy as np
import os
import cv2

     |████████████████████████████████| 727kB 17.9MB/s 
     |████████████████████████████████| 194kB 58.3MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 1.2MB 58.0MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
Mounted at /content/gdrive


In [ ]:
creds = '{"username":"dhrumilshah3","key":"d2d4fbb44fa50e88a37963b6aa08b19f"}'

In [ ]:
cred_path = Path('~/.kaggle/kaggle.json').expanduser()

if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

from kaggle import api

In [ ]:
path = URLs.path('human_proteins_atlas')
Path.BASE_PATH = path

In [ ]:
if not path.exists():
    path.mkdir(parents=true)
    api.competition_download_cli('human-protein-atlas-image-classification', path=path)
    file_extract(path/'human-protein-atlas-image-classification.zip')

  0%|          | 5.00M/17.5G [00:00<08:53, 35.2MB/s]

100%|██████████| 17.5G/17.5G [07:20<00:00, 42.6MB/s]


# Prepare Datasets and Dataloaders

In [ ]:
path_train = path/'train'
path_test = path/'test'
path_labels = path/'train.csv'
path_submission = path/'sample_submission.csv'

## Get the csv data from their paths

In [ ]:
labels = pd.read_csv(path_labels)
submission = pd.read_csv(path_submission)

In [ ]:
labels.head()

In [ ]:
submission.head()

## One Hot Encoding since Multi-Labelled Data

In [ ]:
label_names = {
    0:  "Nucleoplasm",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes",   
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}

In [ ]:
for key in label_names.keys():
  labels[label_names[key]] = 0 #create a column for each value from the category list, and set its value as 0
labels.head(2)

In [ ]:
def encode(row): #pass in 1 row from the entire df
  row.Target = np.array(row.Target.split(" ")).astype(np.int) #turn the target for that row from a string into an array where the items are ints
  for num in row.Target: #iterate over each number in targets array
    name = label_names[int(num)] #that number is a key, so find its corresponding value from categories list
    row.loc[name] = 1 #find the column in that row with the same title as the category and make its value 1
  return row

labels = labels.apply(encode, axis=1)
labels.head(2)

In [ ]:
for key in label_names.keys():
  submission[label_names[key]] = 0 #create a column for each key, and set its value as 0
submission.head(2)

## Get Images for the IDs

In [ ]:
def open_rgby(path,id): #a function that reads RGBY image
    colors = ['red','green','blue','yellow']
    flags = cv2.IMREAD_GRAYSCALE
    img = [cv2.imread(os.path.join(path, id+'_'+color+'.png'), flags).astype(np.float32)/255
           for color in colors]
    return np.stack(img, axis=-1)

In [ ]:
for Id in labels["Id"].head():
    image_stack_array=open_rgby(path_train, Id)
    image_stack_array_processed=np.array(Image.fromarray((image_stack_array * 255).astype(np.uint8)).resize((256, 256)).convert('RGB'))
    bob = Image.fromarray(image_stack_array_processed)
bob
##i need to figure out a way to add these to the training data set

In [ ]:
train_x = labels["Id"].copy(deep=True)

In [ ]:
for id in train_x:
  image_stack_array=open_rgby(path_train, id)
  image_stack_array_processed=np.array(Image.fromarray((image_stack_array * 255).astype(np.uint8)).resize((256, 256)).convert('RGB'))
  train_x["Image"] = Image.fromarray(image_stack_array_processed)